In [33]:
import pandas as pd
import mysql.connector as mysql
import re
import numpy as np

db = mysql.connect(
    host="localhost",
    user="root",
    password="password",
    port="3306",
    database="serlo"
)


def query(sql):
    c = db.cursor()
    c.execute(sql)
    return c.fetchall()


def comment_on_own_page(user_id):
    user_comments = query('''select comment_vote.user_id
    from comment_vote
    inner join comment
    on comment.id = comment_vote.comment_id
    where comment_vote.user_id = comment.author_id''' )

    for user in user_comments:
        commenter_id = user[0]
        if user_id == commenter_id:
            return True 
    return False
    
    
df = pd.read_sql("""
    Select
        user.id,
        user.username,
        user.description,
        user.date,
        sum(events.edits) as edits,
        sum(events.reviews) as reviews,
        sum(events.comments) as comments,
        sum(events.taxonomy) as taxonomy
    From (
        SELECT
            IF(event_log.event_id = 5, 1, 0) as edits,
            IF(event_log.event_id in (6,11), 1, 0) as reviews,
            IF(event_log.event_id in (8,9,14,16), 1, 0) as comments,
            IF(event_log.event_id in (1,2,12,15,17), 1, 0) as taxonomy,
            event_log.actor_id
        FROM event_log
        WHERE event_log.event_id IN (5,6,11,8,9,14,16,1,2,12,15,17)
    ) events
    right join user on user.id = events.actor_id
    group by user.id
""", db)
df.fillna(0, inplace=True)
df.head()
df["comment_on_own_page"] = df["id"].map(comment_on_own_page)  
df_potential_bot = df[
    ((df["edits"] + df["reviews"] + df["taxonomy"] == 0) & (df["comments"] > 0) & df["comment_on_own_page"])
].copy()
df_potential_bot.sort_values(["id"], inplace=True)
df_potential_bot


/Users/kellyshor/.asdf/installs/python/3.9.1/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,username,description,date,edits,reviews,comments,taxonomy,comment_on_own_page
1856,39586,Beachy,NULL,2015-04-24 09:01:24,0.0,0.0,1.0,0.0,True
2250,47567,TheGenius,NULL,2015-10-04 09:22:25,0.0,0.0,1.0,0.0,True
2573,51931,rmg,,2016-02-24 14:25:17,0.0,0.0,2.0,0.0,True
2661,54733,WolfOliver,NULL,2016-03-18 10:33:11,0.0,0.0,3.0,0.0,True
2990,61630,amitsulu,NULL,2016-08-30 06:49:01,0.0,0.0,3.0,0.0,True
...,...,...,...,...,...,...,...,...,...
15463,210033,Unbekannter21,NULL,2021-05-13 14:45:15,0.0,0.0,0.0,0.0,True
15550,212449,claramax600,NULL,2021-05-26 06:45:05,0.0,0.0,0.0,0.0,True
15612,213407,magda,NULL,2021-06-06 13:05:20,0.0,0.0,0.0,0.0,True
15690,215701,fatilass,NULL,2021-06-21 18:26:30,0.0,0.0,0.0,0.0,True
